In [1]:
import matplotlib.pyplot as plt

import numpy as np 
import scipy
from scipy.special import roots_legendre, factorial
from functools import partial

from einops import repeat

In [2]:
def ReLU(x, n):
    # ReLU activation function
    # n : degree 
    return np.maximum(0,x) ** n 

In [19]:
def GP1D(l,n):
    x = np.linspace(0,1,n)
    y = np.linspace(0,1,n)
    mean = np.zeros_like(x)
    gram = np.exp(-np.abs(x.reshape(-1,1) - y.reshape(1,-1))**2/(2*l**2))
    f = np.random.multivariate_normal(mean, gram)
    return f

In [54]:
def Kernel(x, y):
    return 0.5*(x+y-np.abs(x-y)) - x*y

In [86]:
N = 5
npts = 1001
x = np.linspace(0,1,npts)
h = x[1]

# eval kernel function
xx, yy = np.meshgrid(x,x)
xx, yy = xx.reshape(-1), yy.reshape(-1)
K = Kernel(xx,yy).reshape(npts, npts)

# eval force functions
fs = []
for i in range(N):
    fs.append(GP1D(0.02,npts))
fs = np.array(fs)

# calc solution functions
us = h* (K @ fs.T).T

In [99]:
class KernelOGA1d:
    def __init__(self, nNeuron, npts, fs, us):
        self.npts = npts
        self.nNeuron = nNeuron 
        
        # coords
        x = np.linspace(0,1,npts)
        h = x[1]
        xx, yy = np.meshgrid(x,x)
        xs, ys = xx.reshape(-1), yy.reshape(-1)
        self.xs = xs
        self.ys = ys
        self.h = h

        # data
        self.fs = fs 
        self.us = us

        # ref kernel
        self.Gref = self.refkernel(xs, ys).reshape(npts, npts)

        # learned kernel
        self.Gk = np.zeros((npts, npts))

        # neural network
        self.act = partial(ReLU, n=2)
        self.W = np.zeros((nNeuron, 2))
        self.Bias = np.zeros((nNeuron, 1))

        # wb search 
        self.nw = 2 
        self.nb = 1001

        # current neuron
        self.k = 0
    
    def refkernel(self,x,y):
        return 0.5*(x+y-np.abs(x-y)) - x*y

    def optimize(self):
        wxk, wyk, bk = self.optimize_wb()
        return wxk, wyk, bk

    def optimize_wb(self):
        wxk, wyk, bk = self.initialguess_wb()
        return wxk, wyk, bk
    
    def initialguess_wb(self):
        Wx = np.linspace(-1,1,self.nw)
        Wy = np.linspace(-1,1,self.nw)
        B = np.linspace(-2,2,self.nb)
        wwx, wwy, bb = np.meshgrid(Wx, Wy, B)
        wwx = wwx.reshape(-1)
        wwy = wwy.reshape(-1)
        bb = bb.reshape(-1)

        npts = self.npts
        h = self.h
        fs = self.fs 
        us = self.us

        loss = []
        for wx, wy, b in zip(wwx, wwy, bb):
            c = wx*self.xs + wy*self.ys + b
            G = self.act(c).reshape(npts, npts) # neural kernel 

            uGs = h * (G @ fs.T).T
            uGks = h * (self.Gk @ fs.T).T 
            rG = h * ((us - uGks) * uGs).sum()
            E = -0.5 * rG **2
            loss.append(E)

        idx = np.argmin(loss)
        wxi, wyi, bi = wwx[idx], wwy[idx], bb[idx]
        return wxi, wyi, bi
    
    

In [100]:
model = KernelOGA1d(65, 1001, fs, us)

In [101]:
model.optimize()

(1.0, 1.0, 2.0)